<a href="https://colab.research.google.com/github/leoliu00529/CodeSnap/blob/master/CNN_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Conv2D, Dense, Flatten, MaxPooling2D, Dropout, Activation
import pickle
import numpy as np


# in_xtrain = open('xtrain', 'rb')
# x_train = pickle.load(in_xtrain)
# in_xtrain.close()

# in_ytrain = open('ytrain', 'rb')
# y_train = pickle.load(in_ytrain)
# in_ytrain.close()

input_shape = (116, 300, 1)

#Model:

model = Sequential()
model.add(Conv2D(200, (2, 2), padding='same', strides=1, 
                 activation='relu', input_shape=input_shape))
model.add(Conv2D(200, (3, 3), activation='relu'))
model.add(Conv2D(200, (4, 4), activation='relu'))
model.add(Conv2D(200, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(4))
model.add(Activation('softmax'))
print(model.summary())
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.RMSprop(lr=0.0001),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          epochs=3,
          verbose=1,
          batch_size=4)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 116, 300, 200)     1000      
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 114, 298, 200)     360200    
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 111, 295, 200)     640200    
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 107, 291, 200)     1000200   
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 53, 145, 200)      0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 1537000)           0         
_________________________________________________________________
dense_4 (Dense)              (None, 100)               153700100 
__________

In [ ]:
!ls

sample_data


In [ ]:
!pip3 install indexer


ERROR: Command errored out with exit status 1: python setup.py egg_info Check the logs for full command output.


In [ ]:
import spellchecker


ModuleNotFoundError: ignored

In [ ]:
from zipfile import ZipFile
zip = ZipFile('xtrain.zip')
zip.extractall()
zip1 = ZipFile('ytrain.zip')
zip1.extractall()


In [ ]:
import keras
from keras import backend as K
import numpy
import nltk
import numpy as np
import re
from gensim.models import Word2Vec
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
from keras.models import Sequential
from keras.layers import Conv2D, Dense, Flatten, MaxPooling2D, Dropout, Activation
import pickle

nltk.download('punkt')
BATCH_SIZE = 50
DENSE_DROP_OUT = 0.25
LEARNING_RATE = 0.0001
EMBEDDING_DIM = 300
EMBEDDING_LENGTH = 116 #Max sentence length. Apply padding when needed 
NUM_CLASSES = 2 #or 3, when dealing with relation that has an inverse


#learning the vocab and word2vec embedding
all_words = []
with open('''train.txt''') as train_file:
  for line in train_file:
      if line != '\n' and not line.startswith('REL:'):
          all_words.append(re.sub(' .*\n', '', line.lower()))
  train_file.close()         
with open('''test_g.txt''') as test_file:
  for line in test_file:
      if line != '\n' and not line.startswith('REL:'):
          all_words.append(re.sub(' .*\n', '', line.lower()))
  test_file.close()
          
all_words = [nltk.word_tokenize(word) for word in all_words]
wordembedding = Word2Vec(all_words, size=300, min_count=1)
vocab = wordembedding.wv.vocab

RELATION_TYPE = 'overlap'
x_train = []
y_train = []
x_test = []
y_test = []

#Longest sentence: 106
#Thus input: 116 * 300
#two helper function to process the input 

# this function appends the a 116 * 300 word embedding to the train (or test) data
def createInstance(x_train, y_train, fs, fl, ts, tl, label, wordSequence):
    array = []
    length = int(112 - len(wordSequence))
    for each in range(0, int(length/2)):
        array.append(np.zeros(300))
    for i, word in enumerate(wordSequence):
        # if i == fs or i == fs + fl or i == ts or i == ts + tl:
        #     array.append(np.zeros(300))
        try: array.append(wordembedding.wv[word])
        except: array.append(np.zeros(300))
    for each in range(0, length - (int(length / 2))):
        array.append(np.zeros(300))
    array.insert(fs, np.zeros(300))
    array.insert(fs + fl, np.zeros(300))
    array.insert(ts, np.zeros(300))
    array.insert(ts + tl, np.zeros(300))
    x_train.append(array)
    if label:
        y_train.append(0)
    else:
        y_train.append(1)


# this function reads a segment of text from the raw data and extract instances 
# of relations that we concern with. In this case, we only concern with 'overlap'
# relation. And since 'overlap' does not have an inverse relation (unlike 
# 'contains' or 'before'), the label would be either [1, 0] or [0, 1]
def Sentence(wholetext, train):
    wholetext = wholetext.splitlines(True)
    wordSequence = []
    relationLine = []
    for textline in wholetext:
        if textline != '\n':
            if not textline.startswith('REL:'):
                wordSequence.append(str(re.sub(' .*\n', '', textline.lower())))
            else:
                relationLine.append(str(re.sub('\n', '', textline.lower())))
    for textline in relationLine:
        if not '''/''' in textline:
            textline = re.sub('rel: ', '', textline.lower())
            parsed_line= re.findall('[0-9][0-9]*', textline)
            fromPos = int(parsed_line[0])
            secondPart = re.sub('.*?;', '', textline, 1)
            toPos = int(re.findall('[0-9][0-9]*', secondPart)[0])
            from_length = re.findall('.*?;', textline)[0].count(',') + 1
            to_length = re.findall('.*?;', textline)[1].count(',') + 1
            label = False
            if re.sub('.*;', '', textline) == RELATION_TYPE:
                label = True
                # createInstance(x_train, y_train, self.wordSequence[fromPos:fromPos + from_length],
                #                self.wordSequence[toPos:toPos + to_length], label)
            if train:
                createInstance(x_train, y_train, fromPos, from_length, toPos, to_length, label, wordSequence)
            else:
                createInstance(x_test, y_test, fromPos, from_length, toPos, to_length, label, wordSequence)


# read the training data. For testing purpose, only deal with first 50000 lines 
# train1 is a tiny sample of the original training data, containging roughly 100 lines

print('Generating training data')

feed = ''
with open('''train.txt''') as f:
  for i, line in enumerate(f):
    if i < 200000:
      if not line.startswith('\n'):
        feed += line
      else:
        if feed != '':
          Sentence(feed, True)
          feed = ''
  f.close()

# read the testing_ground_truth data
# test1 is a tiny sample of the original testing data, containging roughly 100 lines

print('Generating testing data')

feed = ''
with open('''test_g.txt''') as f:
  for i, line in enumerate(f):
    if i < 1500:
      if not line.startswith('\n'):
        feed += line
      else:
        if feed != '':
          Sentence(feed, False)
          feed = ''
  f.close()

  

# dump all unnecessary data into dist to save spaces
file_wordembedding = open('word', 'wb')
pickle.dump(wordembedding, file_wordembedding)
file_wordembedding.close()

# reshape the data into tensor and adding a dummy dimension
print('Reshaping data')

x_train = np.array(x_train)
print('x_train: ')
print(type(x_train))
print(x_train.shape)

x_train = x_train.reshape(x_train.shape[0], 116, 300, 1)
y_train = keras.utils.to_categorical(y_train, 2)

# file_xtrain = open('xtrain', 'wb')
# pickle.dump(x_train, file_xtrain)
# file_xtrain.close()


x_test = np.array(x_test)
print('x_test: ')
print(type(x_test))
print(x_test.shape)
print(x_test.shape)
x_test = x_test.reshape(x_test.shape[0], 116, 300, 1)
y_test = keras.utils.to_categorical(y_test, 2)


# After this point, all four inputs should be tensors in the form of 
# (sample_num, 116, 300, 1)

input_shape = (EMBEDDING_LENGTH, EMBEDDING_DIM, 1)



# The following paragraph is directly paraphrased from the paper:

# In the CNN-based model, the
# embedding layer is followed by a convolution layer
# that applies convolving filters of sizes (2, 3, 4, 5) with 200 filters each to
# extract n-gram-like features, which are then pooled
# by a max-pooling layer.
# Output of the max-pool layer is fed into a fully connected
# dense layer (drop out rate 0.25), which is followed by the final softmax
# layer outputting the probability distribution over
# the three possible classes for the input. (RMSprop optimizer)

# Some assumptions I have to made: stride size (I assume 1 for conv layer and
# 2 for pooling layer), activation (I assume relu), 
#loss (I assume sparse crossentropy)


# Building the CNN 


print('Building the model')
model = Sequential()

# Four 2D convolution layers
model.add(Conv2D(200, (2, 2), padding='same', strides=1, 
                 activation='relu', input_shape=input_shape))
model.add(Conv2D(200, (3, 3), activation='relu'))
model.add(Conv2D(200, (4, 4), activation='relu'))
model.add(Conv2D(200, (5, 5), activation='relu'))

# One max pooling 
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())

# Dense layer with dropout = 0.25
model.add(Dense(100, activation='relu'))
model.add(Dropout(DENSE_DROP_OUT))
model.add(Dense(2))
model.add(Activation('softmax'))

print('Compiling the model')
print(model.summary())
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.RMSprop(lr=LEARNING_RATE),
              metrics=['accuracy'])
print('Running the model')
model.fit(x_train, y_train,
          epochs=3,
          verbose=1,
          batch_size=2,
          validation_data=(x_test, y_test))

Using TensorFlow backend.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Generating training data
Generating testing data
Reshaping data
x_train: 
<class 'numpy.ndarray'>
(20591, 116, 300)
x_test: 
<class 'numpy.ndarray'>
(149, 116, 300)
(149, 116, 300)
Building the model
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Compiling the model
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 116, 300, 200)     1000      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 114, 298, 200)     360200    
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 111, 295, 200)     640200    
____________________

KeyboardInterrupt: ignored